In [2]:
using Revise

In [3]:
using Plots

[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
[ Info: Precompiling IJuliaExt [2f4121a4-3b3a-5ce6-9c5e-1f2673ce168a]


In [4]:
using Pkg
Pkg.instantiate()

Precompiling project...
  ✓ Plots → FileIOExt
  ✓ Plots → UnitfulExt
  ✓ LammpsAnalysis
  3 dependencies successfully precompiled in 27 seconds. 251 already precompiled.


In [5]:
using HPSAnalysis

### Meta Data

In [6]:
push!(LOAD_PATH, "/uni-mainz.de/homes/ywitzky/phdscripts/Julia/Analysis/")

push!(LOAD_PATH, "/uni-mainz.de/homes/ywitzky/phdscripts/Julia/HPSAnalysis/")
push!(LOAD_PATH, "/uni-mainz.de/homes/ywitzky/phdscripts/Julia/HPSAnalysis/src/")

#push!(LOAD_PATH, "/uni-mainz.de/homes/ywitzky/phdscripts/Analysis/Lammps/")
#import Setup 
using ProteinSequences, Setup

In [7]:
OneToLamba  = Dict([('E', 0.460), ('K',0.514)])
OneToSigma  = Dict([('E', 5.92), ('K',6.36)])
OneToCharge = Dict([('E',-1.0),('K',1.0)])

BoxLength=240
NewPath = "/localscratch/HPS_DATA/HREMD/ArashReproduction_WeakDamp//"
RECALC=true

true

In [8]:
ToCreate =["EKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEK"]


1-element Vector{String}:
 "EKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEKEK"

### Creating Simulations

In [9]:
if false
ProteinNames = []
RelativePaths = []
println(ToCreate)
NewPath = "/localscratch/HPS_DATA/HREMD/ArashReproduction_WeakDamp/"
for (protID, seq) in enumerate(ToCreate)
    protein = "EKV_$protID"
    push!(ProteinNames, "$(protein)")
    append!(RelativePaths, ["./$(protein)/"])

    mkpath(NewPath*"$(protein)/")
    mkpath(NewPath*"$(protein)/Restart/")
    Path = NewPath*"$(protein)/"
    cd(Path)

    N = length(seq)
    Pos = zeros(Float64, (N,3))
    for i in 1:N
        Pos[i, 1] = -BoxLength./2.0+i*3.8
    end

    Info ="Trying to replicate the paper \"Effect of Charge Distribution on the Dynamics of Polyampholytic Disordered Proteins\".\n\n"

    BoxSize = [-BoxLength/2., BoxLength/2.,-BoxLength/2., BoxLength/2.,-BoxLength/2., BoxLength /2.]

    SimName = "$(protein)"

    Setup.writeStartConfiguration("./$(protein)","./$(SimName)_Start", Info, [seq], BoxSize , 100_000_000; SimulationType="ArashModell" , Temperature=300,  InitStyle="Pos", Pos=Pos, OneToChargeDef=OneToCharge, OneToLambdaDef=OneToLamba, OneToSigmaDef=OneToSigma, WriteOutFreq=100 )
    #end
end
mkpath(NewPath*"SlurmScripts/")
Setup.writeCollectedSlurmScript(NewPath*"SlurmScripts/", ProteinNames, RelativePaths ,ones(Int16,length(ProteinNames)),2*ones(Int16,length(ProteinNames)); ProteinsPerSlurmFile=48, Restarts=0, SlurmAccName="rsproteins", JobName="EKV")

end

### Analysis

In [10]:
using LsqFit, Plots, SpecialFunctions, Parquet2, DataFrames
using Parquet2: Dataset

[ Info: Precompiling Parquet2 [98572fba-bba0-415d-956f-fa77e587d26d]
[ Info: Precompiling BangBangDataFramesExt [d787bcad-b5c5-56bb-adaa-6bfddb178a59]
[ Info: Precompiling TransducersDataFramesExt [cefb4096-3352-5e5f-8501-71f024082a88]


In [ ]:

NLag =100000
if false
    tau  = zeros(length(ToCreate))
    Δtau = zeros(length(ToCreate))
    RG   = zeros(length(ToCreate))
    ΔRG  = zeros(length(ToCreate))
    REE  = zeros(length(ToCreate))
    ΔREE = zeros(length(ToCreate))
    for (protID, seq) in enumerate(ToCreate)
        prot = HPSAnalysis.initData(NewPath*"EKV_$protID/", "./EKV_$(protID).lmp"; StepFrequency=1, Reparse=false, LoadAll=true) ;
        prot.EquilibrationTime=10_000
        
        HPSAnalysis.computeCOM!(prot)
        HPSAnalysis.computeRGComponentSeries!(prot)
        HPSAnalysis.computeRGCorrelationTime(prot, NLag)
        HPSAnalysis.plotRGAutocorr(prot)


        HPSAnalysis.computeREESeries!(prot)
        decorTime  = HPSAnalysis.computeREEAutocorr(prot,NLag)
        τ,β,Δτ,Δβ = fitKWK(prot)
        println("$(τ),$(β),$(Δτ),$(Δβ)")
        #plotRGAutocorr(prot, parm)
        tau[protID] = τ/β*gamma(1.0/β)
        Δtau[protID] = sqrt( (Δτ/β*gamma(1/β))^2 + Δβ*τ/β^3*gamma(1/β)*(β+polygamma(0,1/β)))


        RG[protID],ΔRG[protID], REE[protID], ΔREE[protID] = computeSizes(prot, decorTime[1,1])

        HPSAnalysis.SaveAllData(prot)
    end
end

energy file step: 1000001 , number of restarts: 0 vs. 1000001 , 1//1
/localscratch/HPS_DATA/HREMD/ArashReproduction_WeakDamp//EKV_1/Trajectory.h5
7119.760745368525,1.0766340329195914,8.804407459663633,0.0021447636726433916
energy file step: 1000001 , number of restarts: 0 vs. 1000001 , 1//1
/localscratch/HPS_DATA/HREMD/ArashReproduction_WeakDamp//EKV_2/Trajectory.h5
7404.3028121045245,0.9897004718283126,6.95493392908983,0.0013746485984251454
energy file step: 1000001 , number of restarts: 0 vs. 1000001 , 1//1
/localscratch/HPS_DATA/HREMD/ArashReproduction_WeakDamp//EKV_3/Trajectory.h5
8107.591899823372,1.0091844080287102,13.089342100524945,0.0024581166042068094
energy file step: 1000001 , number of restarts: 0 vs. 1000001 , 1//1
/localscratch/HPS_DATA/HREMD/ArashReproduction_WeakDamp//EKV_4/Trajectory.h5
4534.17814470399,0.8648704044862077,4.9340163208023,0.0012053011016607484
energy file step: 1000001 , number of restarts: 0 vs. 1000001 , 1//1


In [11]:
NewPath = "/localscratch/HPS_DATA/HREMD/ArashReproduction_WeakDamp/"
protID=1
prot = LammpsAnalysis.initData(NewPath*"EKV_$protID/", "./EKV_$(protID).lmp"; StepFrequency=1, Reparse=false, LoadAll=true) ;


In [12]:
using StaticArrays

In [13]:
size(prot.x)

(50, 1000001)

In [46]:
function test(prot)
    CMaskx = MArray{Tuple{50,1}, Float32}(zeros(50))
    res = zeros(prot.NSteps)
    for i in 1:prot.NSteps
        CMaskx .= @view prot.x[:,i]
        res[i] = sum(CMaskx, dims=1)[1,1]
    end
    return res
end

test (generic function with 2 methods)

In [ ]:
function test_alt(prot)
    CMaskx = MArray{Tuple{50,1}, Float32}(zeros(50))
    res = zeros(prot.NSteps)
    for i in 1:prot.NSteps
        #CMaskx .= @view prot.x[:,i]
        res[i] = sum(prot.x[1:N], dims=1)
    end
    return res
end

In [71]:
function test_old(prot, res)
    CMaskx = MArray{Tuple{50,1}, Float32}(zeros(50))
    for (chain, start, stop) in zip(1:prot.NChains, prot.ChainStart, prot.ChainStop)
        res .= sum(prot.x[start:stop,:], dims=1)
    end
    return res
end

test_old (generic function with 2 methods)

In [91]:
function test_new(prot, res)
    for i in axes(prot.x, 2)
        res[i] = sum(@view prot.x[:,i])
    end
    return res
end

test_new (generic function with 1 method)

In [92]:
res = zeros(1,prot.NSteps)

@time test_new(prot, res)
@time test_new(prot, res)

  0.574089 seconds (8.05 M allocations: 171.240 MiB, 5.03% gc time, 8.94% compilation time)
  0.559133 seconds (8.00 M allocations: 167.816 MiB, 11.05% gc time)


1×1000001 Matrix{Float64}:
 -1155.0  -1154.55  -1150.62  -1146.9  …  -1332.42  -1329.13  -1326.26

In [72]:
res = zeros(1,prot.NSteps)

@time test_old(prot, res)
@time test_old(prot, res)

  0.143657 seconds (42.64 k allocations: 197.469 MiB, 8.24% gc time, 25.90% compilation time)
  0.113660 seconds (11 allocations: 194.551 MiB, 8.07% gc time)


1×1000001 Matrix{Float64}:
 -1155.0  -1154.55  -1150.62  -1146.9  …  -1332.42  -1329.13  -1326.26

In [51]:
using ProfileView, Profile

[ Info: Precompiling ProfileView [c46f51b8-102a-5cf2-8d2c-8597cb0e0da7]


In [63]:
@time test(prot)
@time test(prot)

  0.961392 seconds (7.00 M allocations: 205.971 MiB, 2.40% gc time)
  1.014201 seconds (7.00 M allocations: 205.971 MiB, 4.15% gc time)


1000001-element Vector{Float64}:
 -1155.0
 -1154.550537109375
 -1150.6207275390625
 -1146.9013671875
 -1140.1151123046875
 -1140.8868408203125
 -1143.145751953125
 -1140.6646728515625
 -1136.853515625
 -1139.97705078125
 -1139.94140625
 -1147.108154296875
 -1146.0931396484375
     ⋮
 -1338.9320068359375
 -1340.6063232421875
 -1575.18017578125
 -1580.1484375
 -1579.2777099609375
 -1340.115966796875
 -1329.777587890625
 -1331.06689453125
 -1331.7659912109375
 -1332.423095703125
 -1329.1322021484375
 -1326.2608642578125

In [32]:
#CMaskx = reshape((@view prot.x[:]),(50,1,1000001))
CMaskx = MArray{Tuple{50,1}, Float32}(zeros(50))
@time CMaskx .= @view prot.x[:,15]
@time bla=MArray{Tuple{50,1}, Float32}(MArray{Tuple{50,1}, Float32}(@view prot.x[:,15]))

  0.000020 seconds (4 allocations: 160 bytes)
  0.000018 seconds (4 allocations: 480 bytes)


50×1 MMatrix{50, 1, Float32, 50} with indices SOneTo(50)×SOneTo(1):
 -114.48452
 -110.86716
 -107.78048
 -103.98989
  -99.8423
  -96.150764
  -92.38892
  -88.666985
  -85.02599
  -81.300026
  -77.28201
  -73.826355
  -69.86911
    ⋮
   28.228256
   31.950687
   35.776417
   39.21366
   42.84701
   46.50735
   49.938477
   53.774338
   57.223022
   60.37103
   63.545372
   66.958824

In [33]:
typeof(CMaskx)

MMatrix{50, 1, Float32, 50} (alias for MArray{Tuple{50, 1}, Float32, 2, 50})

In [34]:
0.000020 * 1000001


20.000020000000003